- renew dataset (version 2)

In [1]:
import os
import warnings
import numpy as np 
import pandas as pd
import lightgbm as lgb
import scipy.stats as stats
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, accuracy_score

warnings.filterwarnings("ignore")

In [2]:
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat - x,2) + np.power(yhat-y,2)) + 15 * np.abs(fhat-f)
    return intermediate.sum() / xhat.shape[0]

In [3]:
ORG_DIR = '/kaggle/input/indoor-location-navigation'
DATA_DIR = '/kaggle/input/indoor-competition-dataset'

sub = pd.read_csv(ORG_DIR+"/sample_submission.csv")
train_files = [i for i in os.listdir(DATA_DIR) if "train" in i]
test_files = [i for i in os.listdir(DATA_DIR) if "test" in i]

all_sites = sorted(set(sub["site_path_timestamp"].apply(lambda x: x.split("_")[0])))
sub = sub.set_index("site_path_timestamp")

# modelling

In [4]:
lgbm_params = {'objective': 'regression', 'metric': 'rmse', 'boosting_type': 'gbdt', 
               'tree_learner': 'serial', 'learning_rate': 0.1, "num_leaves": 30, 
               'random_seed':44, 'verbose': -1}

lgbm_params_f = {'objective': 'multiclass', 'metric': 'multi_logloss',
                     'boosting_type': 'gbdt','tree_learner': 'serial',
                     'learning_rate': 0.1, "num_leaves": 30, 
               'random_seed':44, 'n_estimators': 5000, 'verbose': -1}
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=224)
predictions = list()

all_f_pred = np.array([])
all_x_pred = np.array([])
all_y_pred = np.array([])
all_f_label = np.array([])
all_x_label = np.array([])
all_y_label = np.array([])

for site in tqdm(all_sites):
    train_data = pd.read_csv(DATA_DIR+"/"+site+str("_train.csv"), index_col="path")
    test_data = pd.read_csv(DATA_DIR+"/"+site+str("_test.csv"))

    train_paths = train_data.index.unique()
    non_targets = list(train_data.iloc[:,:-4].columns)

    # predict floor first
    X = train_data[non_targets+["f"]]
    X_test = test_data[non_targets]

    label_f = np.array([])
    valid_f_pred = np.array([])
    test_f_pred = np.zeros([len(test_data), n_folds])
    for fold, (tr_group_idx, val_group_idx) in enumerate(kf.split(train_paths)):
        tr_paths, val_paths = train_paths[tr_group_idx], train_paths[val_group_idx]
    
        X_train = X[X.index.isin(tr_paths)].drop("f", axis=1).reset_index(drop=True)
        X_valid = X[X.index.isin(val_paths)].drop("f", axis=1).reset_index(drop=True)
        y_train = X[X.index.isin(tr_paths)]["f"].reset_index(drop=True)
        y_valid = X[X.index.isin(val_paths)]["f"].reset_index(drop=True)
    
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
        model = lgb.LGBMClassifier(**lgbm_params_f)
        model.fit(X_train, y_train,
                       eval_set=[(X_valid, y_valid)],
                       eval_metric='multi_logloss',
                       verbose=False,
                       early_stopping_rounds=20
                       )
    
        tmp_pred = model.predict(X_valid).astype(int)
        valid_f_pred = np.concatenate([valid_f_pred, tmp_pred])
        label_f = np.concatenate([label_f, y_valid])
    
        test_f_pred[:, fold] = model.predict(X_test).astype(int)

    preds_f_mode = stats.mode(test_f_pred, axis=1)
    test_f_pred = preds_f_mode[0].astype(int).reshape(-1)
    accuracy = accuracy_score(label_f, valid_f_pred)

    # predict x based on floor information
    X = train_data[non_targets+["x"]]
    X_test = test_data[non_targets]
    X["f_pred"] = valid_f_pred
    X_test["f_pred"] = test_f_pred 

    label_x = np.array([])
    valid_x_pred = np.array([])
    test_x_pred = np.zeros(len(test_data))
    for fold, (tr_group_idx, val_group_idx) in enumerate(kf.split(train_paths)):
        tr_paths, val_paths = train_paths[tr_group_idx], train_paths[val_group_idx]
    
        X_train = X[X.index.isin(tr_paths)].drop("x", axis=1).reset_index(drop=True)
        X_valid = X[X.index.isin(val_paths)].drop("x", axis=1).reset_index(drop=True)
        y_train = X[X.index.isin(tr_paths)]["x"].reset_index(drop=True)
        y_valid = X[X.index.isin(val_paths)]["x"].reset_index(drop=True)
    
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
        modelx = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
                num_boost_round=10000,early_stopping_rounds=20,verbose_eval = 1000)
    
        tmp_pred = modelx.predict(X_valid, num_iteration = modelx.best_iteration)
        valid_x_pred = np.concatenate([valid_x_pred, tmp_pred])
        label_x = np.concatenate([label_x, y_valid])
    
        test_x_pred += modelx.predict(X_test, num_iteration = modelx.best_iteration) / n_folds
    
    score_x = np.sqrt(mean_squared_error(label_x, valid_x_pred))

    # predict x based on floor information
    X = train_data[non_targets+["y"]]
    X_test = test_data[non_targets]
    X["f_pred"] = valid_f_pred
    X_test["f_pred"] = test_f_pred

    label_y = np.array([])
    valid_y_pred = np.array([])
    test_y_pred = np.zeros(len(test_data))
    for fold, (tr_group_idx, val_group_idx) in enumerate(kf.split(train_paths)):
        tr_paths, val_paths = train_paths[tr_group_idx], train_paths[val_group_idx]
    
        X_train = X[X.index.isin(tr_paths)].drop("y", axis=1).reset_index(drop=True)
        X_valid = X[X.index.isin(val_paths)].drop("y", axis=1).reset_index(drop=True)
        y_train = X[X.index.isin(tr_paths)]["y"].reset_index(drop=True)
        y_valid = X[X.index.isin(val_paths)]["y"].reset_index(drop=True)
    
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
        
        modely = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
                num_boost_round=10000,early_stopping_rounds=20,verbose_eval = 1000)
    
        tmp_pred = modely.predict(X_valid, num_iteration = modely.best_iteration)
        valid_y_pred = np.concatenate([valid_y_pred, tmp_pred])
        label_y = np.concatenate([label_y, y_valid])
    
        test_y_pred += modely.predict(X_test, num_iteration = modely.best_iteration) / n_folds
    
    score_y = np.sqrt(mean_squared_error(label_y, valid_y_pred))
    print("each score:", accuracy, score_x, score_y)
    print("site:", site, comp_metric(valid_x_pred, valid_y_pred, valid_f_pred, label_x, label_y, label_f))

    test_preds = pd.DataFrame(np.stack((test_f_pred, test_x_pred, test_y_pred))).T
    test_preds.columns = sub.columns
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)
    
    all_x_pred = np.concatenate([all_x_pred, valid_x_pred])
    all_x_label = np.concatenate([all_x_label, label_x])
    all_y_pred = np.concatenate([all_y_pred, valid_y_pred])
    all_y_label = np.concatenate([all_y_label, label_y])
    all_f_pred = np.concatenate([all_f_pred, valid_f_pred])
    all_f_label = np.concatenate([all_f_label, label_f])
print("overall: ", comp_metric(all_x_pred, all_y_pred, all_f_pred, all_x_label, all_y_label, all_f_label))

  0%|          | 0/24 [00:00<?, ?it/s]

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[327]	training's rmse: 1.71463	valid_1's rmse: 10.493
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[451]	training's rmse: 1.31596	valid_1's rmse: 12.4532
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[234]	training's rmse: 2.06665	valid_1's rmse: 11.8231
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[197]	training's rmse: 2.27454	valid_1's rmse: 19.0309
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[151]	training's rmse: 2.69999	valid_1's rmse: 10.0711
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[434]	training's rmse: 0.968353	valid_1's rmse: 6.10878
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is

# submission

In [5]:
all_preds = pd.concat(predictions)
all_preds = all_preds.sort_values("site_path_timestamp")
all_preds.to_csv('submission.csv')